# Efficient Coding Assistant with simpleaichat

Many coders use ChatGPT for coding help, however the web interface can be slow and contain unnecessary discussion when you want code. With some system prompt engineering and simplechatapi streaming, you can cut down code time generation and costs significantly.

**DISCLAIMER: Your mileage may vary in terms of code quality and accuracy in practice, but this is a good, hackable starting point.**

In [1]:
!pip install -q simpleaichat

from simpleaichat import AIChat
from getpass import getpass

For the following cell, input your OpenAI API key when prompted. **It will not be saved to the notebook**.

In [2]:
api_key = getpass("OpenAI Key: ")

OpenAI Key: ··········


In [3]:
params = {"temperature": 0.0}  # for reproducibility
model = "gpt-3.5-turbo"  # in production, may want to use model="gpt-4" if have access

ai = AIChat(api_key=api_key, console=False, params=params, model=model)

Let's start with a simple `is_palindrome()` function in Python, and track how long it takes to run. The output of this should be similar to what is shown in the ChatGPT webapp.

In [4]:
%%time
response = ai("Write an is_palindrome() function in Python.")
print(response)

Sure, here's an example implementation of an `is_palindrome()` function in Python:

```python
def is_palindrome(s):
    """
    Returns True if the given string s is a palindrome, False otherwise.
    """
    # Remove any non-alphanumeric characters and convert to lowercase
    s = ''.join(c for c in s if c.isalnum()).lower()
    # Check if the string is equal to its reverse
    return s == s[::-1]
```

This function takes a string `s` as input and returns `True` if it is a palindrome (i.e. reads the same forwards and backwards), and `False` otherwise. The function first removes any non-alphanumeric characters from the string and converts it to lowercase, then checks if the resulting string is equal to its reverse using Python's slice notation (`[::-1]` returns the string in reverse order).
CPU times: user 77.6 ms, sys: 4.01 ms, total: 81.6 ms
Wall time: 14.2 s


That's the typical implementation. However, there's a trick to cut the processing time in half, well known by technical hiring managers who want to trip up prospective candidates.

ChatGPT outputs the statistically most common implementation, but it's not necessairily the best. A second pass allows ChatGPT to refine its output.

In [5]:
%%time
response = ai("Make it more efficient.")
print(response)

Here's an optimized version of the `is_palindrome()` function that avoids creating a new string and uses a loop to compare the characters at each end of the string:

```python
def is_palindrome(s):
    """
    Returns True if the given string s is a palindrome, False otherwise.
    """
    # Convert to lowercase and remove non-alphanumeric characters
    s = ''.join(c.lower() for c in s if c.isalnum())
    # Compare characters at each end of the string
    for i in range(len(s) // 2):
        if s[i] != s[-i-1]:
            return False
    return True
```

This version of the function still removes non-alphanumeric characters and converts the string to lowercase, but instead of creating a new string to check for equality with its reverse, it uses a loop to compare the characters at each end of the string. The loop only needs to iterate over half of the string (using `len(s) // 2`), since the other half is already being compared by the loop. If any pair of characters is not equal, the 

In [6]:
ai.total_length

704

In all, it took ~33 seconds and utilized 704 tokens. But there's a lot of unnecessary natter in the output:

- The conversational preamble before the code
- Docstrings and code comments
- A long explanation of the code which may be redundant to the above

All this natter adds latency and cost.

There's a few techniques we can do with simpleaichat especially with the system prompt to mitigate these concerns, and therefore get a massive speedup:

- Tell ChatGPT to ONLY generate code. Due to its RLHF training it sometimes doesn't always abide by it.
- Tell ChatGPT to not output code comments.
- Add a `stop` parameter to stop generating when the code encounters the final 3 backticks, skipping the explanation. Due to tokenization these could be followed by a space or a newline.

Now, for the new `system` prompt:

In [7]:
system_optimized = """You are a Python code example.

Follow ALL the following rules:
- ONLY EVER RESPOND WITH CODE IN PYTHON MARKDOWN BLOCKS, AND NOTHING ELSE
- NEVER include code comments."""

new_params = {
    "temperature": 0.0,
    "stop": ["``` ", "```\n"]
}

ai_2 = AIChat(api_key=api_key, system=system_optimized, model=model, params=new_params)

In [8]:
%%time
response = ai_2("is_palindrome")
print(response)

```python
def is_palindrome(s):
    return s == s[::-1]
```
CPU times: user 13 ms, sys: 3.07 ms, total: 16 ms
Wall time: 1.81 s


In [9]:
%%time
response = ai_2("Make it more efficient.")
print(response)

```python
def is_palindrome(s):
    length = len(s)
    for i in range(length // 2):
        if s[i] != s[length - i - 1]:
            return False
    return True

CPU times: user 24.4 ms, sys: 2.44 ms, total: 26.9 ms
Wall time: 3.84 s


In [14]:
ai_2.total_length

197

~6 seconds total with 197 tokens used: that's 5x faster at 1/3 the cost!

## Create a Function

Now we can create a function to automate the two calls we did above for any arbitrary input.

For each call, we'll create an independent temporary session within simpleaichat and then clean it up. We'll also use a regex to strip unneded backticks.

In [10]:
from uuid import uuid4
import re

ai_func = AIChat(api_key=api_key, console=False)
pattern = r"```python\n|```$"
def gen_code(query):
    id = uuid4()
    ai_func.new_session(api_key=api_key, id=id, system=system_optimized, params=new_params, model=model)
    _ = ai_func(query, id=id)
    response_optimized = ai_func("Make it more efficient.", id=id)
    
    response_cleaned = re.sub(pattern, "", response_optimized).strip()
    
    ai_func.delete_session(id=id)
    return response_cleaned

In [11]:
%%time
code = gen_code("is_palindrome")
print(code)

def is_palindrome(word):
    for i in range(len(word)//2):
        if word[i] != word[-i-1]:
            return False
    return True
CPU times: user 35.4 ms, sys: 1.07 ms, total: 36.4 ms
Wall time: 5.13 s


In [13]:
%%time
code = gen_code("reverse string")
print(code)

string = "hello world"
reverse_string = ''.join(reversed(string))
print(reverse_string)
CPU times: user 29 ms, sys: 2.19 ms, total: 31.2 ms
Wall time: 4.04 s


In [14]:
%%time
code = gen_code("fibonacci")
print(code)

def fibonacci(n):
    if n <= 1:
        return n
    else:
        a, b = 0, 1
        for i in range(1, n):
            a, b = b, a + b
        return b
CPU times: user 44.5 ms, sys: 5.46 ms, total: 49.9 ms
Wall time: 7.91 s


In [15]:
%%time
code = gen_code("async fibonacci")
print(code)

async def fibonacci(n: int, memo={}) -> int:
    if n in memo:
        return memo[n]
    if n <= 1:
        return n
    memo[n] = await fibonacci(n-1, memo) + await fibonacci(n-2, memo)
    return memo[n]
CPU times: user 43.2 ms, sys: 10.6 ms, total: 53.9 ms
Wall time: 8.72 s


In [16]:
%%time
code = gen_code("multiprocess hash")
print(code)

import hashlib
import multiprocessing

def hash_string(string):
    return hashlib.sha256(string.encode()).hexdigest()

if __name__ == '__main__':
    strings = ['hello', 'world', 'python', 'multiprocessing']
    with multiprocessing.Pool() as pool:
        hashes = pool.map_async(hash_string, strings)
        hashes.wait()
        result = hashes.get()
    print(result)
CPU times: user 64.8 ms, sys: 4.9 ms, total: 69.7 ms
Wall time: 11.9 s


## MIT License

Copyright (c) 2023 Max Woolf

Permission is hereby granted, free of charge, to any person obtaining a copy
of this software and associated documentation files (the "Software"), to deal
in the Software without restriction, including without limitation the rights
to use, copy, modify, merge, publish, distribute, sublicense, and/or sell
copies of the Software, and to permit persons to whom the Software is
furnished to do so, subject to the following conditions:

The above copyright notice and this permission notice shall be included in all
copies or substantial portions of the Software.

THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR
IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY,
FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE
AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER
LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM,
OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE
SOFTWARE.
